While there are records of instances of school shootings in the United States dating back to the Enoch Brown school massacre in Greencastle, PA on July 26, 1764, the first mass school shooting according to the modern convention of counting a mass school shooting as being a three person or more causualty on the actual school grounds did not occur until March 26, 1893 in Plain Dealing, LA during an evening dance at the school.  According to Wikipedia beween then and 1966 only three other mass school shottings occured.  They were as follows:

* The December 13, 1898 Charleston, WV school exhibition incident killing 6 and wounding at least 1 other
* The May 6, 1940 Pasadena, CA - South Pasadena Junior High School - murders leaving 5 dead and 2 others wounded
* The February 2, 1960 Hartford City, IN - William Reed School - shooting killing 3

This means that only four modern day standard mass school shootings occured before 1966 even though around 130 school-related shootings are on record with Wikipedia from this early time period in America history.  Thankfully, the rest of these horrific events do not meet the modern definition of being a mass school shooting by today's standards as outlined above.

In 1968 & 1970, a total of 9 student protesters were killed and another 48 were injured at the hands of United States government officers.  Those two years the following three major mass incidents occured included the disgraceful South Carolina State University racist protesters in Orangeburg, SC on February 8, 1968, the Kent State University Vietnam War protesters being gunned down by armed National Guard solders on May 4, 1970 in Kent, OH, and when the police shot the protestors at the Jackson State University in Jackson, MS over the United States' military presence in Cambodia on May 15, 1970 (occuring only eleven days following the Kent State massacre).

While, the Columbine High School massacre did not happen until 1999, these attacks by the United States government and the August 1, 1966 University of Texas - Austin massacre in Austin, Texas killing 17 and wounding 31 paired with the November 12, 1966 (roughly 3.5 months after the University of Texas massacre) in Mesa, AZ where 5 died and another 2 were injured, set off a new type of shock value for the vast increase number killed in any particular incident.

Now, in the first month and a half of 2018, the United States not only has had the deadliest mass school shooting in United States history, but has had the school shooting rate increase to 1 occuring in every 60 hours.  Many National Rifle Association (NRA) backed politicans continue to look the other way or actively try to stall any and all gun-control reform or legislation.

While it is clear that gun violence is not possible without guns, the interpretation of the *Second Amendment* of *The Bill of Rights* to the *United States Constitution* protects at least some of her citizens to bare arms under certain well-regulated circumstances as follows: "A well regulated Militia, being necessary to the security of a free State, the right of the people to keep and bear Arms, shall not be infringed."  These restiction of being first well-regulated, governed under a lawful collective body, and being protected only at times of necessity of securing a free State not only is the law, but also there in this the Founders brilliance which lies both the pearls of wisdom and the rub as to any ambiguity.  It is clear from the wording of this ammendment verses all the others to-date that the Founders signing the Constitution clearly intended it to have breathing room in case of a foreign invasion, civil war, hunting, or any other communal benefit as escribed by the federal or local government as it is their most basic duty to The People to pretect and defend the country and all her citizens.

Having said that, I believe we as a society have run a stray regarding gun mentality.  In the days of the "Wild West," salon and high noon shootouts occurred far too frequently.  As did armed bandits, robbers, and criminal behaviors we have tried desparately to outlaw ever since including from the 1920 mobster crime family sprees -- sometimes to what appears no to little avail.  That having been said, the National Rifle Association (NRA) has over extended its duties to protect the 2nd Amendment, and has in the words of my former professor and Noble Prize winner, Noam Chomsky, created an illegal "Profit Over People" enterprise.  Under the current state of the NRA, politicians (mostly Republican) have been bought off by this special interest group with campaign contributions, bribes, and disinformation campaigns as reported by most of the world's most major media news outlets.

Therefore, it is my intention to look into these claims and see if the data shows increases of mass gun violence in either low gun control states or in states where U.S. congressional members NRA ratings are higher.  If this data does show this then the legislative solution might favor changes to states alone on a state-by-state basis.  If not, the data might then mean that due to being a free interstate society that stricter gun laws must come top down from the Federal government in order to solve the gun violence epidemic sweeping every state of the country.  One thing is for certain, change can not come fast enough, but over regulation historically leads to an increase in crime and eventual reversal of sensability.

Like any Python project, it is always best practice to put all your import libraries at the top of a file both for easy access and documenting reference; as well as, for the functional purpose of ensuring the file has properly loaded the libraries prior to their need within the program itself.

In [1]:
# Import Libraries

import json
from csv import DictReader
from io import BytesIO
from sqlite3 import connect, Error, version
from urllib.request import build_opener, urlopen
from zipfile import ZipFile
from http import HTTPStatus
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openpyxl import load_workbook

Next, let's retrieve the first dataset necessary for the project via the internet -- the *Stanford MSA Database*.

In [2]:
# Unzip Stanford MSA Database CSV file

msa_url = urlopen("https://stanford.box.com/shared/static/75lmj9rvw9mhtfqfjijhcc6oa15o5m4a.zip")
zipfile = ZipFile(BytesIO(msa_url.read()))
zipfile.namelist() # Proves proper connection occured

['MSA/',
 'MSA/Stanford_MSA_Data_Dictionary (V.5).pdf',
 '__MACOSX/',
 '__MACOSX/MSA/',
 '__MACOSX/MSA/._Stanford_MSA_Data_Dictionary (V.5).pdf',
 'MSA/Stanford_MSA_Database_for_release_06142016.xlsx',
 '__MACOSX/MSA/._Stanford_MSA_Database_for_release_06142016.xlsx']

Now lets transverse the file structure to the appropriate worksheet.

In [3]:
# Load Workbook & Proper Worksheet

workbook = load_workbook(zipfile.extract('MSA/Stanford_MSA_Database_for_release_06142016.xlsx','store', pwd=None))
worksheet = workbook['Published']
HTTPStatus.OK.description

'Request fulfilled, document follows'

Now while we could use the current Excel file format, lets convert the file to being in Comma Separated Values (CSV) format for ease of use later on.

In [4]:
# Convert Excel file to Comma Separated Values file

msa_xls = pd.read_excel('store/MSA/Stanford_MSA_Database_for_release_06142016.xlsx', 'Published', index_col=None)
msa_xls.to_csv('store/MSA/Stanford_MSA_Database_for_release.csv', encoding='utf-8', index=False)

Now lets create the SQL database connection.

In [5]:
# Create a SQL database connection

conn = connect(':memory:')
print(version)

conn.text_factory = str  # allows utf-8 data to be stored

c = conn.cursor()

table_name = "Shootings"

with open('store/MSA/Stanford_MSA_Database_for_release.csv', newline='') as csv_file:
    msa_reader = DictReader(csv_file, delimiter=",", dialect='excel', quotechar='"')

    header = True
    for row in msa_reader:
        try:
            if header:
                # gather column names from the first row of the csv
                header = False

                sql = f"DROP TABLE IF EXISTS {table_name}"
                c.execute(sql)

                column_name_correction = []
                print([column_name_correction.append(column.replace(' ', '_') for column in row)])
                print(", ".join([f"{column} text" for column in column_name_correction]))
                sql = f"CREATE TABLE {table_name} ({', '.join(['%s text' % column for column in column_name_correction])})"
                c.execute(sql)
                # print(c.execute(sql))

                for column in row:
                    if column.lower().endswith("_id"):
                        index = f"{table_name}__{column}"
                        sql = f"CREATE INDEX {index} on {table_name} ({column})"
                        c.execute(sql)
                        # print(c.execute(sql))

            insert_sql = f"INSERT INTO {table_name} VALUES ({', '.join(['?' for column in row])})"
            # print(insert_sql)

            row_length = len(row)
            # else:
            #     # skip lines that don't have the right number of columns
            #     if len(row) == row_length:
            #         c.execute(insert_sql, row)
        except Error as error:
            exit(f'file {csv_file}, line {msa_reader.line_num}: {error}')
            print(error)

    conn.commit()


2.6.0
[None]
<generator object <genexpr> at 0x0BD84D80> text
near "<": syntax error


While every Congress is unique, within the follwing file is the makeup of the 2015 - 2016 U.S. Congress as reported by *The Washington Post*.

In [6]:
# File for Web Scrapping & Parsing

gun_legislation_url = 'https://www.washingtonpost.com/graphics/national/gun-legislation/'
gun_legislation_html_soup = BeautifulSoup(urlopen(gun_legislation_url), 'html.parser')
stats_json_string = gun_legislation_html_soup.find('script', attrs={'src': 'js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540'})
print(stats_json_string)

<script charset="utf-8" src="js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540" type="text/javascript"></script>


Having discovered where the data lives, we then must convert the JavaScript file ready for parsing.  In this case we cut off the top and bottom code sections, then replace the unquoted keywords to being double-quoted per the JSON library's requirement for parsing.

In [7]:
full_gun_legislation_url = 'https://www.washingtonpost.com/graphics/national/gun-legislation/js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540'
legislation_request = requests.get(full_gun_legislation_url)

try:
    # print(legislation_request.headers)
    b = legislation_request.text.find("{first:")
    e = legislation_request.text.find("{}],2:")
    scrape = legislation_request.text[b - 1:e - 2]

    scrape = scrape.replace('first:', '"first":')
    scrape = scrape.replace('last:', '"last":')
    scrape = scrape.replace('fullLast:', '"fullLast":')
    scrape = scrape.replace('party:', '"party":')
    scrape = scrape.replace('house:', '"house":')
    scrape = scrape.replace('state:', '"state":')
    scrape = scrape.replace('control:', '"control":')
    scrape = scrape.replace('rights:', '"rights":')
    scrape = scrape.replace('id:', '"id":')
    scrape = scrape.replace('score:', '"score":')
    scrape = scrape.replace('pi:', '"pi":')
    scrape = scrape.replace('grade:', '"grade":')
    scrape = scrape.replace('gi:', '"gi":')

    # scrape = json.loads(scrape)
    print(scrape)

except Error as error:
    legislation_request.status_code

[{"first":"Ralph","last":"Abraham","fullLast":"Abraham","party":"r","house":"h","state":"LA","control":[],"rights":[59,62,64,70,74,75,90,94,84],"id":0,"score":9,"pi":1,"grade":"AQ","gi":13},{"first":"Alma","last":"Adams","fullLast":"Adams","party":"d","house":"h","state":"NC","control":[2,9,10,14,15,17,31,37,40,50,53],"rights":[],"id":1,"score":-11,"pi":0,"grade":"F","gi":0},{"first":"Robert","last":"Aderholt","fullLast":"Aderholt","party":"r","house":"h","state":"AL","control":[],"rights":[64,71,78,90,94],"id":2,"score":5,"pi":1,"grade":"A","gi":1},{"first":"Pete","last":"Aguilar","fullLast":"Aguilar","party":"d","house":"h","state":"CA","control":[9,10,14,50,53],"rights":[],"id":3,"score":-5,"pi":1,"grade":"F","gi":1},{"first":"Rick","last":"Allen","fullLast":"Allen","party":"r","house":"h","state":"GA","control":[],"rights":[55,57,59,62,63,64,65,71,74,78,84,90,95],"id":4,"score":13,"pi":0,"grade":"A","gi":0},{"first":"Mark","last":"Amodei","fullLast":"Amodei","party":"r","house":"h"

Closes SQL Database Connections

In [8]:
# Close the SQL connection to our SQLite database

c.close()
conn.close()